## Multi layer perceptron

In [2]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

Metemos a todas las imágenes en un batch

In [3]:
imgs = MNIST.images()
X = hcat(float.(reshape.(imgs, :))...) |> gpu

┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /opt/julia/packages/Flux/dkJUV/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    619      0 --:--:-- --:--:-- --:--:--   619
100 9680k  100 9680k    0     0  1489k      0  0:00:06  0:00:06 --:--:-- 2144k
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /opt/julia/packages/Flux/dkJUV/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    663      0 --:--:-- --:--:-- --:--:--   662
100 28881  100 28881    0     0  18766      0  0:00:01  0:00:01 --:--:--  180k
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /opt/julia/packages/Flux/dkJUV/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   T

784×60000 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

#### One hot Encoding de las labels

In [4]:
labels = MNIST.labels()
Y = onehotbatch(labels, 0:9) |> gpu


10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

Creamos EL perecptrón

In [5]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

Chain(Dense(784, 32, NNlib.relu), Dense(32, 10), NNlib.softmax)

In [6]:
loss(x, y) = crossentropy(m(x), y)


loss (generic function with 1 method)

In [7]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))


accuracy (generic function with 1 method)

In [8]:
dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

### Entenamos

In [9]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))


loss(X, Y) = 2.3569255f0 (tracked)
loss(X, Y) = 1.4901104f0 (tracked)
loss(X, Y) = 0.9029282f0 (tracked)
loss(X, Y) = 0.6057667f0 (tracked)
loss(X, Y) = 0.4829521f0 (tracked)
loss(X, Y) = 0.41564733f0 (tracked)
loss(X, Y) = 0.37467605f0 (tracked)
loss(X, Y) = 0.34605557f0 (tracked)
loss(X, Y) = 0.32393768f0 (tracked)
loss(X, Y) = 0.30612007f0 (tracked)
loss(X, Y) = 0.29125106f0 (tracked)
loss(X, Y) = 0.27854028f0 (tracked)


### Medimos Accuracy

In [10]:
accuracy(X, Y)

tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

accuracy(tX, tY)

0.9256